# Benchmark of edsger.shortestpath.path_length

https://www.timlrx.com/2019/05/05/benchmark-of-popular-graph-network-packages/

[Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/index.html)

In [1]:
import os

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

from edsger.shortestpath import convert_sorted_graph_to_csr, path_length
from edsger.commons import INFINITY_PY, UITYPE_PY

rs = 124
np.random.seed(rs)
data_dir_path = '../../data/'

| test case | NetworkX loading | NetworkX sssp | Edsger loading | Edsger sssp 1 |
|-----------|-----------------:|--------------:|---------------:|--------------:|
| Amazon    |   5.13 |  1.47 | 0.03 | 0.14 |
| Google    |  21.60 |  5.21 | 0.12 | 0.55 |
| Pokec     | 125.00 | 39.90 | 0.55 | 2.69 |


##  Amazon product co-purchasing network, March 02 2003

https://snap.stanford.edu/data/amazon0302.html

Nodes: 262111   
Edges: 1234877

In [2]:
network_file_path = os.path.join(data_dir_path, "Amazon0302.txt")

In [3]:
amazon = pd.read_csv(network_file_path, sep='\t', skiprows=3, header=0)
amazon.columns = ['tail_vert', 'head_vert']
# amazon['cost'] = 1.
amazon['cost'] = np.random.rand(len(amazon))
amazon.head(2)

,tail_vert,head_vert,cost
0,0,1,0.106065
1,0,2,0.745471


In [4]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234877 entries, 0 to 1234876
Data columns (total 3 columns):
tail_vert    1234877 non-null int64
head_vert    1234877 non-null int64
cost         1234877 non-null float64
dtypes: float64(1), int64(2)
memory usage: 28.3 MB


In [5]:
n_vertices = 262111

### NetworkX

In [6]:
# %%time
# G = nx.from_pandas_edgelist(amazon, 'tail_vert', 'head_vert', ['cost'], create_using=nx.DiGraph)

In [7]:
# %%time
# cost_nx = nx.algorithms.single_source_dijkstra_path_length(G, 0, weight='cost')

In [8]:
# %%time
# cost_nx_df = pd.DataFrame(data=cost_nx.values(), index=cost_nx.keys(), columns=['path_length']).sort_index()
# cost_nx_df.head(2)

### Edsger

In [9]:
# amazon.sort_values(by=['tail_vert', 'head_vert'], ascending=True, inplace=True)
assert amazon.index.is_monotonic
assert amazon.index.is_unique
assert amazon.index.min() == 0
assert amazon.index.max() + 1 == len(amazon)

In [10]:
%%time
tail_vert = amazon.tail_vert.values.astype(UITYPE_PY)
head_vert = amazon.head_vert.values.astype(UITYPE_PY)
edge_weights = amazon.cost.values
indptr = convert_sorted_graph_to_csr(tail_vert, head_vert, n_vertices)

CPU times: user 24.7 ms, sys: 4.14 ms, total: 28.9 ms
Wall time: 27.8 ms


In [11]:
# %%time
# np.insert(amazon.tail_vert.value_counts().sort_index().reindex(np.arange(n_vertices), fill_value=0).cumsum().values, 0, 0)

In [12]:
%%time
cost_ed = path_length(head_vert, indptr, edge_weights, 0, n_vertices, n_jobs=-1)

CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 143 ms


In [13]:
# cost_ed_df = pd.DataFrame(data=cost_ed, columns=['path_length'])

### Check

In [14]:
# cost_nx_df.equals(cost_ed_df)

## Google web graph

https://snap.stanford.edu/data/web-Google.html
    
Nodes: 875713  
Edges: 5105039  

In [15]:
network_file_path = os.path.join(data_dir_path, "web-Google.txt")
google = pd.read_csv(network_file_path, sep='\t', skiprows=3, header=0)
google.columns = ['tail_vert', 'head_vert']
# google['cost'] = 1.
google['cost'] = np.random.rand(len(google))
google.head(2)

,tail_vert,head_vert,cost
0,0,11342,0.252119
1,0,824020,0.135259


In [16]:
google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5105039 entries, 0 to 5105038
Data columns (total 3 columns):
tail_vert    int64
head_vert    int64
cost         float64
dtypes: float64(1), int64(2)
memory usage: 116.8 MB


In [17]:
n_vertices = 875713

### NetworkX

In [18]:
# %%time
# G = nx.from_pandas_edgelist(google, 'tail_vert', 'head_vert', ['cost'], create_using=nx.DiGraph)

In [19]:
# G.number_of_nodes()

In [20]:
# G.number_of_edges()

In [21]:
# %%time
# cost_nx = nx.algorithms.single_source_dijkstra_path_length(G, 0, weight='cost')

In [22]:
# %%time
# cost_nx_df = pd.DataFrame(data=cost_nx.values(), index=cost_nx.keys(), columns=['path_length']).sort_index()
# cost_nx_df.head(2)

In [23]:
# len(cost_nx_df)

### Edsger

In [24]:
google.sort_values(by=['tail_vert', 'head_vert'], ascending=True, inplace=True)
google.reset_index(inplace=True, drop=True)
assert google.index.is_monotonic
assert google.index.is_unique
assert google.index.min() == 0
assert google.index.max() + 1 == len(google)

In [25]:
print(google.tail_vert.min(), google.tail_vert.max())
print(google.head_vert.min(), google.head_vert.max())

0 916427
0 916427


In [26]:
%%time
tail_vert = google.tail_vert.values.astype(UITYPE_PY)
head_vert = google.head_vert.values.astype(UITYPE_PY)
edge_weights = google.cost.values
indptr = convert_sorted_graph_to_csr(tail_vert, head_vert, 916428)

CPU times: user 73.8 ms, sys: 40.5 ms, total: 114 ms
Wall time: 112 ms


In [27]:
%%time
cost_ed = path_length(head_vert, indptr, edge_weights, 0, 916428, n_jobs=-1)

CPU times: user 607 ms, sys: 42 µs, total: 607 ms
Wall time: 603 ms


In [28]:
# cost_ed_df = pd.DataFrame(data=cost_ed, columns=['path_length'])

In [29]:
# cost_ed_df = cost_ed_df[cost_ed_df.path_length < INFINITY_PY]

In [30]:
# len(cost_ed_df)

### Check

In [31]:
# cost_nx_df.equals(cost_ed_df)

## Pokec social network

https://snap.stanford.edu/data/soc-Pokec.html

Nodes: 1632803  
Edges: 30622564  

In [32]:
network_file_path = os.path.join(data_dir_path, "soc-pokec-relationships.txt")
pokec = pd.read_csv(network_file_path, sep='\t', skiprows=3, header=0)
pokec.columns = ['tail_vert', 'head_vert']
# pokec['cost'] = 1.
pokec['cost'] = np.random.rand(len(pokec))
pokec.head(2)

,tail_vert,head_vert,cost
0,1,4,0.516708
1,1,5,0.762884


In [33]:
pokec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30622560 entries, 0 to 30622559
Data columns (total 3 columns):
tail_vert    int64
head_vert    int64
cost         float64
dtypes: float64(1), int64(2)
memory usage: 700.9 MB


In [34]:
n_vertices = 1632803

### NetworkX

In [35]:
# %%time
# G = nx.from_pandas_edgelist(pokec, 'tail_vert', 'head_vert', ['cost'], create_using=nx.DiGraph)

In [36]:
# %%time
# cost_nx = nx.algorithms.single_source_dijkstra_path_length(G, 1, weight='cost')

In [37]:
# %%time
# cost_nx_df = pd.DataFrame(data=cost_nx.values(), index=cost_nx.keys(), columns=['path_length']).sort_index()
# cost_nx_df.head(2)

### Edsger

In [38]:
pokec.sort_values(by=['tail_vert', 'head_vert'], ascending=True, inplace=True)
pokec.reset_index(inplace=True, drop=True)
assert pokec.index.is_monotonic
assert pokec.index.is_unique
assert pokec.index.min() == 0
assert pokec.index.max() + 1 == len(pokec)

In [39]:
print(pokec.tail_vert.min(), pokec.tail_vert.max())
print(pokec.head_vert.min(), pokec.head_vert.max())

1 1632803
1 1632803


In [40]:
%%time
tail_vert = pokec.tail_vert.values.astype(UITYPE_PY)
head_vert = pokec.head_vert.values.astype(UITYPE_PY)
edge_weights = pokec.cost.values
indptr = convert_sorted_graph_to_csr(tail_vert, head_vert, 1632804)

CPU times: user 338 ms, sys: 217 ms, total: 554 ms
Wall time: 554 ms


In [41]:
%%time
cost_ed = path_length(head_vert, indptr, edge_weights, 1, 1632804, n_jobs=-1)

CPU times: user 2.95 s, sys: 4.89 ms, total: 2.96 s
Wall time: 2.94 s
